In [4]:
!pip install yfinance


  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 42.4 MB/s eta 0:00:00
  Installing build dependencies ... one
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 68.4 MB/s eta 0:00:00
  DEPRECATION: Building 'multitasking' using the legacy setup.py bdist_wheel mechanism, which will be removed in a future version. pip 25.3 will enforce this behaviour change. A possible replacement is to use the standardized build interface by setting the `--use-pep517` option, (possibly combined with `--no-build-isolation`), or adding a `pyproject.toml` file to the source tree of 'multitasking'. Discussion can be found at https://github.com/pypa/pip/issues/6334
  Created wheel for multitasking: filename=multitasking-0.0.12-py3-none-any.whl size=15548 sha256=2e19ed33257905f56f8eb8cafb282953d8113ff1b7614a04d0d03803797d8353
  Stored in directory: /Users/hv/Libra

In [7]:
!pip install pandas numpy scikit-learn yfinance matplotlib joblib



In [8]:
import pandas as pd
import numpy as np

import yfinance as yf

from sklearn.model_selection import TimeSeriesSplit
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, roc_auc_score

import joblib


In [9]:
ticker = "AAPL"          # BIST için örn: "THYAO.IS"
df = yf.download(ticker, start="2018-01-01", progress=False)

df = df.dropna()
df.head()


/var/folders/0q/m9286zn93lqbp1wnnhp6n9pw0000gn/T/ipykernel_3686/1777138467.py:2: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(ticker, start="2018-01-01", progress=False)


Price,Close,High,Low,Open,Volume
Ticker,AAPL,AAPL,AAPL,AAPL,AAPL
Date,,,,,
2018-01-02,40.341888,40.351258,39.639313,39.850088,102223600
2018-01-03,40.334866,40.878192,40.271636,40.405124,118071600
2018-01-04,40.522221,40.625266,40.299739,40.407465,89738400
2018-01-05,40.983566,41.070216,40.526893,40.618228,94640000
2018-01-08,40.831341,41.126421,40.732977,40.831341,82271200


In [11]:
type(df.columns), df.columns


(pandas.core.indexes.multi.MultiIndex,
 MultiIndex([( 'Close', 'AAPL'),
             (  'High', 'AAPL'),
             (   'Low', 'AAPL'),
             (  'Open', 'AAPL'),
             ('Volume', 'AAPL')],
            names=['Price', 'Ticker']))

In [12]:
type(df["Close"]), df["Close"].head()


(pandas.core.frame.DataFrame,
 Ticker           AAPL
 Date                 
 2018-01-02  40.341888
 2018-01-03  40.334866
 2018-01-04  40.522221
 2018-01-05  40.983566
 2018-01-08  40.831341)

In [13]:
close = df["Close"]
if isinstance(close, pd.DataFrame):
    close = close.iloc[:, 0]   # ilk sütunu al (Series'e indir)


In [14]:
data = df.copy()

data["ret_1"] = close.pct_change()

data["ma_10"] = close.rolling(10).mean()
data["ma_50"] = close.rolling(50).mean()
data["close_over_ma10"] = close / data["ma_10"] - 1
data["close_over_ma50"] = close / data["ma_50"] - 1

data["vol_10"] = data["ret_1"].rolling(10).std()
data["vol_20"] = data["ret_1"].rolling(20).std()

data["vol_ratio"] = data["Volume"] / data["Volume"].rolling(20).mean()

data = data.dropna()
data.tail()


Price,Close,High,Low,Open,Volume,ret_1,ma_10,ma_50,close_over_ma10,close_over_ma50,vol_10,vol_20,vol_ratio
Ticker,AAPL,AAPL,AAPL,AAPL,AAPL,,,,,,,,
Date,,,,,,,,,,,,,
2025-12-11,278.029999,279.589996,273.809998,279.100006,33248000,-0.002690,280.365002,267.295979,-0.008328,0.040158,0.008662,0.009630,0.763604
2025-12-12,278.279999,279.220001,276.820007,277.899994,39532900,0.000899,280.308002,267.723959,-0.007235,0.039429,0.008526,0.009608,0.918571
2025-12-15,274.109985,280.149994,272.839996,280.149994,50409100,-0.014985,279.409000,268.050757,-0.018965,0.022605,0.007780,0.010241,1.167248
2025-12-16,274.609985,275.500000,271.790009,272.820007,37648600,0.001824,278.250998,268.414128,-0.013085,0.023083,0.006347,0.009266,0.879275
2025-12-17,271.839996,276.160004,271.690002,275.010010,49406521,-0.010087,277.019998,268.726296,-0.018699,0.011587,0.006569,0.009612,1.148875


In [15]:
df.head()


Price,Close,High,Low,Open,Volume
Ticker,AAPL,AAPL,AAPL,AAPL,AAPL
Date,,,,,
2018-01-02,40.341888,40.351258,39.639313,39.850088,102223600
2018-01-03,40.334866,40.878192,40.271636,40.405124,118071600
2018-01-04,40.522221,40.625266,40.299739,40.407465,89738400
2018-01-05,40.983566,41.070216,40.526893,40.618228,94640000
2018-01-08,40.831341,41.126421,40.732977,40.831341,82271200


In [17]:
data["ret_fwd_1"] = data["Close"].pct_change().shift(-1)  # yarının getirisi
data["y"] = (data["ret_fwd_1"] > 0).astype(int)

data = data.dropna()
data[["Close","ret_fwd_1","y"]].tail()


Price,Close,ret_fwd_1,y
Ticker,AAPL,,
Date,,,
2025-12-10,278.779999,-0.002690,0
2025-12-11,278.029999,0.000899,1
2025-12-12,278.279999,-0.014985,0
2025-12-15,274.109985,0.001824,1
2025-12-16,274.609985,-0.010087,0


In [19]:
list(data.columns)



[('Close', 'AAPL'),
 ('High', 'AAPL'),
 ('Low', 'AAPL'),
 ('Open', 'AAPL'),
 ('Volume', 'AAPL'),
 ('ret_1', ''),
 ('ma_10', ''),
 ('ma_50', ''),
 ('ret_fwd_1', ''),
 ('y', '')]

In [20]:
feature_cols = ["ret_1","close_over_ma10","close_over_ma50","vol_10","vol_20","vol_ratio"]
missing = [c for c in feature_cols if c not in data.columns]
print("Missing:", missing)


Missing: ['close_over_ma10', 'close_over_ma50', 'vol_10', 'vol_20', 'vol_ratio']


In [21]:
data = df.copy()

# Close'u garanti Series 
close = data["Close"]
if isinstance(close, pd.DataFrame):
    close = close.iloc[:, 0]

# 1) günlük getiri
data["ret_1"] = close.pct_change()

# 2) hareketli ortalama oranları
data["ma_10"] = close.rolling(10).mean()
data["ma_50"] = close.rolling(50).mean()
data["close_over_ma10"] = close / data["ma_10"] - 1
data["close_over_ma50"] = close / data["ma_50"] - 1

# 3) volatilite
data["vol_10"] = data["ret_1"].rolling(10).std()
data["vol_20"] = data["ret_1"].rolling(20).std()

# 4) hacim trendi
data["vol_ratio"] = data["Volume"] / data["Volume"].rolling(20).mean()

# hedef (yarın up/down)
data["ret_fwd_1"] = close.pct_change().shift(-1)
data["y"] = (data["ret_fwd_1"] > 0).astype(int)

data = data.dropna()
data.tail()


Price,Close,High,Low,Open,Volume,ret_1,ma_10,ma_50,close_over_ma10,close_over_ma50,vol_10,vol_20,vol_ratio,ret_fwd_1,y
Ticker,AAPL,AAPL,AAPL,AAPL,AAPL,,,,,,,,,,
Date,,,,,,,,,,,,,,,
2025-12-10,278.779999,279.750000,276.440002,277.750000,33038300,0.005772,280.317001,266.839431,-0.005483,0.044748,0.008616,0.009740,0.745813,-0.002690,0
2025-12-11,278.029999,279.589996,273.809998,279.100006,33248000,-0.002690,280.365002,267.295979,-0.008328,0.040158,0.008662,0.009630,0.763604,0.000899,1
2025-12-12,278.279999,279.220001,276.820007,277.899994,39532900,0.000899,280.308002,267.723959,-0.007235,0.039429,0.008526,0.009608,0.918571,-0.014985,0
2025-12-15,274.109985,280.149994,272.839996,280.149994,50409100,-0.014985,279.409000,268.050757,-0.018965,0.022605,0.007780,0.010241,1.167248,0.001824,1
2025-12-16,274.609985,275.500000,271.790009,272.820007,37648600,0.001824,278.250998,268.414128,-0.013085,0.023083,0.006347,0.009266,0.879275,-0.010087,0


In [22]:
feature_cols = ["ret_1","close_over_ma10","close_over_ma50","vol_10","vol_20","vol_ratio"]
X = data[feature_cols]
y = data["y"]
X.shape, y.value_counts()


((1952, 6),
 y
 1    1048
 0     904
 Name: count, dtype: int64)

In [23]:
split_idx = int(len(data) * 0.8)  # %80 train, %20 test

X_train, X_test = X.iloc[:split_idx], X.iloc[split_idx:]
y_train, y_test = y.iloc[:split_idx], y.iloc[split_idx:]

X_train.shape, X_test.shape


((1561, 6), (391, 6))

In [24]:
pipe = Pipeline([
    ("scaler", StandardScaler()),
    ("model", LogisticRegression(max_iter=2000))
])

pipe.fit(X_train, y_train)


Pipeline(steps=[('scaler', StandardScaler()),
                ('model', LogisticRegression(max_iter=2000))])

In [28]:
proba = pipe.predict_proba(X_test)[:, 1]
pred = (proba > 0.55).astype(int)

print("ROC-AUC:", roc_auc_score(y_test, proba))
print(classification_report(y_test, pred))


ROC-AUC: 0.44558745348219037
              precision    recall  f1-score   support

           0       0.42      0.78      0.55       171
           1       0.48      0.15      0.23       220

    accuracy                           0.43       391
   macro avg       0.45      0.47      0.39       391
weighted avg       0.45      0.43      0.37       391



In [26]:
joblib.dump(pipe, "model.joblib")
print("Saved model.joblib")


Saved model.joblib


In [27]:
# ticker = "THYAO.IS"
# degisen satir bu yeni data